# MNIST with CNN

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Helper Functions

Function to help intialize random weights for fully connected or convolutional layers, we leave the shape attribute as a parameter for this.

In [4]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

Same as init_weights, but for the biases

In [5]:
def init_bias(shape):
    init_bias_vals = tf.constant(shape =shape, value = 0.1)
    return tf.Variable(init_bias_vals)

Create a 2D convolution using builtin conv2d from TF. From those docs:

Computes a 2-D convolution given 4-D `input` and `filter` tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
and a filter / kernel tensor of shape
`[filter_height, filter_width, in_channels, out_channels]`, this op
performs the following:

1. Flattens the filter to a 2-D matrix with shape
   `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual*
   tensor of shape `[batch, out_height, out_width,
   filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch
   vector.


In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, 
                        strides = [1, 1, 1, 1], 
                        padding = 'SAME')

Create a max pooling layer, again using built in TF functions:

Performs the max pooling on the input.

    Args:
      value: A 4-D `Tensor` with shape `[batch, height, width, channels]` and
        type `tf.float32`.
      ksize: A list of ints that has length >= 4.  The size of the window for
        each dimension of the input tensor.
      strides: A list of ints that has length >= 4.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. 

In [7]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, 
                          ksize = [1, 2, 2, 1],
                          strides = [1, 2, 2, 1], 
                          padding = 'SAME')

Using the conv2d function, we'll return an actual convolutional layer here that uses an ReLu activation.

In [8]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

This is a normal fully connected layer

In [9]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### Placeholders

In [10]:
x = tf.placeholder(tf.float32, shape = [None,784])

In [11]:
y_true = tf.placeholder(tf.float32, shape = [None,10])

### Layers

In [12]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [13]:
# Using a 2 by 2 filter
# You can change the 32 output, that essentially represents the amount of filters used
# You need to pass in 32 to the next input though, the 1 comes from the original input of 
# a single image.
convo_1 = convolutional_layer(x_image,
                              shape = [2, 2 , 1, 32])
convo_1_pooling = max_pool_2by2(convo_1)

In [14]:
# Using a 2 by 2 filter 
# You can actually change the 64 output if you want, you can think of that as a representation
# of the amount of 6by6 filters used.
convo_2 = convolutional_layer(convo_1_pooling,
                              shape = [3, 3, 32, 64])
convo_2_pooling = max_pool_2by2(convo_2)

In [15]:
# Why 7 by 7 image? Because we did 2 pooling layers, so (28/2)/2 = 7
# 64 then just comes from the output of the previous Convolution
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7 * 7 * 64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [16]:
# Dropout placeholder
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, 
                                 keep_prob = hold_prob)

In [17]:
y_pred = normal_full_layer(full_one_dropout, 10)

### Loss Function

In [18]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_true,
                                                                          logits = y_pred))

### Optimizer

In [19]:
# Training with the Adam Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001)
train = optimizer.minimize(cross_entropy)

### Intialize Variables

In [20]:
init = tf.global_variables_initializer()

### Session

In [21]:
steps = 5000

with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        # Training on batches of 50
        batch_x , batch_y = mnist.train.next_batch(50)
        
        # Training with dropout = 0.5
        sess.run(train,
                 feed_dict = {x : batch_x,
                              y_true : batch_y,
                              hold_prob : 0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Matches
            matches = tf.equal(tf.argmax(y_pred, 1),
                               tf.argmax(y_true, 1))

            # tf.cast is to given datatype
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            # Calculate the accuracy
            # Hold probability = 1 is same as dropout = 0.
            print(sess.run(acc, feed_dict = {x : mnist.test.images, 
                                             y_true : mnist.test.labels,
                                             hold_prob : 1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.1544


Currently on step 100
Accuracy is:
0.7519


Currently on step 200
Accuracy is:
0.832


Currently on step 300
Accuracy is:
0.8829


Currently on step 400
Accuracy is:
0.8962


Currently on step 500
Accuracy is:
0.9137


Currently on step 600
Accuracy is:
0.9209


Currently on step 700
Accuracy is:
0.9279


Currently on step 800
Accuracy is:
0.9316


Currently on step 900
Accuracy is:
0.9348


Currently on step 1000
Accuracy is:
0.942


Currently on step 1100
Accuracy is:
0.941


Currently on step 1200
Accuracy is:
0.9468


Currently on step 1300
Accuracy is:
0.9465


Currently on step 1400
Accuracy is:
0.9492


Currently on step 1500
Accuracy is:
0.9522


Currently on step 1600
Accuracy is:
0.9563


Currently on step 1700
Accuracy is:
0.9577


Currently on step 1800
Accuracy is:
0.9598


Currently on step 1900
Accuracy is:
0.9594


Currently on step 2000
Accuracy is:
0.9637


Currently on step 2100
Accuracy is:
0.9614


Currently on step 2200
Ac

## Great Job!